# 17: Production Pipeline Testing

## Setup & Validation

In [1]:
import sys
import json
import pandas as pd
import numpy as np
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

sys.path.append('../')

print("🟢 Basic imports complete")

🟢 Basic imports complete


In [2]:
# VALIDATE IMPORTS - FAIL FAST IF SOMETHING'S WRONG
print("Validating module structure...\n")

errors = []

# Check each import
try:
    from src.predictors.qualifying_predictor import QualifyingPredictor
    print("🟢 QualifyingPredictor")
except ImportError as e:
    errors.append(f"🔴 QualifyingPredictor: {e}")
    print(f"🔴 QualifyingPredictor: {e}")

try:
    from src.predictors.race_predictor import RacePredictor
    print("🟢 RacePredictor")
except ImportError as e:
    errors.append(f"🔴 RacePredictor: {e}")
    print(f"🔴 RacePredictor: {e}")

try:
    from src.predictors.driver_predictor import DriverRanker
    print("🟢 DriverRanker")
except ImportError as e:
    errors.append(f"🔴 DriverRanker: {e}")
    print(f"🔴 DriverRanker: {e}")

try:
    from src.pipelines.learning_system import LearningSystem
    print("🟢 LearningSystem")
except ImportError as e:
    errors.append(f"🔴 LearningSystem: {e}")
    print(f"🔴 LearningSystem: {e}")

try:
    from src.utils.performance_tracker import PerformanceTracker
    print("🟢 PerformanceTracker")
except ImportError as e:
    errors.append(f"🔴 PerformanceTracker: {e}")
    print(f"🔴 PerformanceTracker: {e}")

# Check underlying dependencies
try:
    from src.extractors.session_extractor import extract_session_order_robust
    print("🟢 session_extractor")
except ImportError as e:
    errors.append(f"🔴 session_extractor: {e}")
    print(f"🔴 session_extractor: {e}")

try:
    from src.predictors.team_predictor import rank_teams_for_track
    print("🟢 team_predictor")
except ImportError as e:
    errors.append(f"🔴 team_predictor: {e}")
    print(f"🔴 team_predictor: {e}")

try:
    from src.predictors.blended_predictor import blend_predictions
    print("🟢 blended_predictor")
except ImportError as e:
    errors.append(f"🔴 blended_predictor: {e}")
    print(f"🔴 blended_predictor: {e}")

try:
    from src.utils.lineup_manager import get_lineups
    print("🟢 lineup_manager")
except ImportError as e:
    errors.append(f"🔴 lineup_manager: {e}")
    print(f"🔴 lineup_manager: {e}")

print("\n" + "="*70)

if errors:
    print("\n🛑 IMPORT VALIDATION FAILED!")
    print("="*70)
    print("\nErrors found:")
    for error in errors:
        print(f"  {error}")
    raise ImportError("Module structure invalid - see errors above")
else:
    print("🟢 ALL IMPORTS VALID - Ready to test!")
    print("="*70)

Validating module structure...

🟢 QualifyingPredictor
🟢 RacePredictor
🟢 DriverRanker
🟢 LearningSystem
🟢 PerformanceTracker
🟢 session_extractor
🟢 team_predictor
🟢 blended_predictor
🟢 lineup_manager

🟢 ALL IMPORTS VALID - Ready to test!


In [3]:
# FastF1 setup
import fastf1 as ff1
ff1.Cache.enable_cache('../data/raw/.fastf1_cache')

print("🟢 FastF1 cache enabled")

🟢 FastF1 cache enabled


## Configuration

In [4]:
SEASON = 2025

# Fetch races dynamically from FastF1
print(f"Fetching {SEASON} season calendar...")
schedule = ff1.get_event_schedule(SEASON)

# Get all race events (exclude testing)
RACES = []
for idx, event in schedule.iterrows():
    event_format = event.get('EventFormat', '')
    if 'testing' not in event_format.lower():
        RACES.append(event['EventName'])

print(f"🟢 Loaded {len(RACES)} races from {SEASON} calendar")
print(f"   First race: {RACES[0]}")
print(f"   Last race: {RACES[-1]}")

Fetching 2025 season calendar...
🟢 Loaded 24 races from 2025 calendar
   First race: Australian Grand Prix
   Last race: Abu Dhabi Grand Prix


## Initialize Components

In [5]:
PROJECT_ROOT = Path.cwd().parent

# --- paths ---
DRIVER_CHAR_PATH = PROJECT_ROOT / "data/processed/testing_files/driver_characteristics/driver_characteristics.json"
TESTING_DIR = PROJECT_ROOT / "data/processed/testing_files"
DATA_DIR = PROJECT_ROOT / "data"

# Track what loaded
loaded, errors_init = [], []

try:
    driver_ranker = DriverRanker(DRIVER_CHAR_PATH)
    loaded.append("driver ranker")
except Exception as e:
    errors_init.append(f"driver ranker: {e}")
    driver_ranker = None

try:
    quali_predictor = QualifyingPredictor(driver_ranker=driver_ranker, data_dir=TESTING_DIR)
    loaded.append("qualifying predictor")
except Exception as e:
    errors_init.append(f"qualifying predictor: {e}")
    quali_predictor = None

try:
    race_predictor = RacePredictor(
        data_dir=DATA_DIR,
        driver_chars=driver_ranker.drivers,          
        driver_chars_path=DRIVER_CHAR_PATH           
    )
    race_predictor.set_uncertainty(new_regs=False)
    loaded.append("race predictor")
except Exception as e:
    errors_init.append(f"race predictor: {e}")
    race_predictor = None

try:
    learning = LearningSystem(data_dir=DATA_DIR)
    loaded.append("learning system")
except Exception as e:
    errors_init.append(f"learning system: {e}")
    learning = None

try:
    tracker = PerformanceTracker(data_dir=DATA_DIR)
    loaded.append("performance tracker")
except Exception as e:
    errors_init.append(f"performance tracker: {e}")
    tracker = None

if loaded:
    print(f"🟢 Loaded: {', '.join(loaded)}")
if errors_init:
    print(f"🛑 Failed: {', '.join(errors_init)}")

if not quali_predictor:
    raise RuntimeError("Cannot proceed without qualifying predictor!")


Loaded characteristics for 27 drivers
🟢 Loaded: driver ranker, qualifying predictor, race predictor, learning system, performance tracker


## Quick Smoke Test - Test on 1 Race First!

In [6]:
print("SMOKE TEST - Testing on first race only")
print("="*70)

test_race = RACES[0]
print(f"\nRace: {test_race}")

try:
    result = quali_predictor.predict(
        year=SEASON,
        race_name=test_race,
        method='session_order',
        verbose=True
    )
    
    print(f"\n🟢 SMOKE TEST PASSED!")
    print(f"   Method: {result['method']}")
    print(f"   Expected MAE: {result['expected_mae']:.2f}")
    print(f"   Grid size: {len(result['grid'])}")
    print(f"\nFirst 3 positions:")
    for p in result['grid'][:3]:
        print(f"  {p['position']}. {p['driver']} ({p['team']})")
    
    print(f"\n🟢 Ready to run full test suite!")
    SMOKE_TEST_PASSED = True
    
except Exception as e:
    print(f"\n🔴 SMOKE TEST FAILED: {e}")
    print(f"\n🛑 DO NOT PROCEED - Fix the error above first!")
    import traceback
    traceback.print_exc()
    SMOKE_TEST_PASSED = False
    raise

SMOKE TEST - Testing on first race only

Race: Australian Grand Prix
   Predicting Australian Grand Prix (normal weekend)
   Method: session_order, Session: FP3
  → Loaded via 'FP3': 10 teams

🟢 SMOKE TEST PASSED!
   Method: session_order_FP3
   Expected MAE: 4.00
   Grid size: 20

First 3 positions:
  1. ANT (Mercedes)
  2. RUS (Mercedes)
  3. SAI (Williams)

🟢 Ready to run full test suite!


## Test 1: Qualifying Predictions

**Only runs if smoke test passed!**

In [7]:
if not SMOKE_TEST_PASSED:
    print("⏭️  SKIPPING - Smoke test failed")
else:
    print("TEST 1: QUALIFYING PREDICTIONS (First 5 races)")
    print("="*70)
    
    # Load actual results
    with open(f'../data/processed/testing_files/validation/{SEASON}_qualifying_results.json') as f:
        actual_results = json.load(f)
    
    # Test methods
    methods_to_test = [
        {'name': 'Session Order', 'method': 'session_order', 'blend_weight': None},
        {'name': 'Blend 50/50', 'method': 'blend', 'blend_weight': 0.5},
    ]
    
    results_by_method = defaultdict(list)
    
    # Test first 5 races only for speed
    for race_name in RACES[:5]:
        if race_name not in actual_results['races']:
            print(f"🛑  {race_name}: No actual results")
            continue
        
        actual = actual_results['races'][race_name]
        
        print(f"\n{race_name}")
        
        for method_config in methods_to_test:
            try:
                prediction = quali_predictor.predict(
                    year=SEASON,
                    race_name=race_name,
                    method=method_config['method'],
                    blend_weight=method_config['blend_weight'] or 0.5,
                    verbose=False
                )
                
                # Compare
                errors = []
                for actual_pos in actual['positions']:
                    driver = actual_pos['driver']
                    actual_p = actual_pos['position']
                    pred = next((p for p in prediction['grid'] if p['driver'] == driver), None)
                    if pred:
                        errors.append(abs(pred['position'] - actual_p))
                
                mae = np.mean(errors) if errors else None
                
                if mae:
                    results_by_method[method_config['name']].append({
                        'race': race_name,
                        'mae': mae
                    })
                    print(f"  🟢 {method_config['name']:<20} MAE: {mae:.2f}")
                
            except Exception as e:
                print(f"  🛑 {method_config['name']:<20} {str(e)[:50]}")
    
    # Summary
    print(f"\n{'='*70}")
    print("SUMMARY")
    print("="*70)
    for method_name, results in results_by_method.items():
        if results:
            avg_mae = np.mean([r['mae'] for r in results])
            print(f"{method_name:<20} {avg_mae:.2f} MAE ({len(results)} races)")

TEST 1: QUALIFYING PREDICTIONS (First 5 races)

Australian Grand Prix
  🟢 Session Order        MAE: 4.00
  🟢 Blend 50/50          MAE: 3.20

Chinese Grand Prix
  🟢 Session Order        MAE: 2.90
  🟢 Blend 50/50          MAE: 2.80

Japanese Grand Prix
  🟢 Session Order        MAE: 3.40
  🟢 Blend 50/50          MAE: 3.60

Bahrain Grand Prix
  🟢 Session Order        MAE: 3.80
  🟢 Blend 50/50          MAE: 3.10

Saudi Arabian Grand Prix
  🟢 Session Order        MAE: 2.90
  🟢 Blend 50/50          MAE: 2.80

SUMMARY
Session Order        3.40 MAE (5 races)
Blend 50/50          3.10 MAE (5 races)


## Validation Report

In [8]:
print("="*70)
print("VALIDATION COMPLETE")
print("="*70)

if SMOKE_TEST_PASSED:
    print("\n🟢 Pipeline works!")
else:
    print("\n🔴 Tests failed - fix errors above")

VALIDATION COMPLETE

🟢 Pipeline works!
